In [1]:
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/Timeline_of_the_2019%E2%80%9320_Wuhan_coronavirus_outbreak'

In [3]:
tables = pd.read_html(url)
# Since the 1st table on Wiki includes three tables, the new_confirmed table is the 4th.

### New Confirmed Cases of Coronavirus in Mainland China by Provincial Divisions

In [4]:
# Since the 1st table on Wiki includes three tables, the new_confirmed table is the 4th.
# Check the infomation on in the Dataframe
df_China_new_confirmed_original = tables[3]
df_China_new_confirmed_original.info()
df_China_new_confirmed_original.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 33 columns):
Date (CST)       22 non-null object
Hubei            18 non-null object
Guangdong        14 non-null object
Beijing          13 non-null object
Shanghai         13 non-null object
Zhejiang         12 non-null object
Tianjin          12 non-null object
Chongqing        12 non-null object
Jiangxi          12 non-null object
Shandong         12 non-null object
Henan            12 non-null object
Hunan            12 non-null object
Sichuan          12 non-null object
Yunnan           11 non-null object
Shanxi           10 non-null object
Fujian           11 non-null object
Liaoning         11 non-null object
Hainan           11 non-null object
Anhui            11 non-null object
Guizhou          11 non-null object
Guangxi          11 non-null object
Ningxia          11 non-null object
Hebei            11 non-null object
Jiangsu          11 non-null object
Jilin            11 non-null obje

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National
0,2020.01.11,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41
1,2020.01.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020.01.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020.01.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020.01.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# remove the last column of the dataframe
df_China_new_confirmed_c = df_China_new_confirmed_original.iloc[:,:-1]
# remove the last 2 rows of the dataframe
df_China_new_confirmed_r = df_China_new_confirmed_c[:-2]
# df_China_new_confirmed_r.tail()

In [6]:
# # Rename the 1st column 
df_China_new_confirmed_r = df_China_new_confirmed_r.rename(columns={"Date (CST)" : "Date"})
# df_China_new_confirmed_r['Date']=df_China_new_confirmed_r['Date'].str.replace(".", "-")

df_China_new_confirmed = df_China_new_confirmed_r.copy()
# convert 'Date' column format to datetime
df_China_new_confirmed['Date'] = pd.to_datetime(df_China_new_confirmed['Date'], format= "%Y-%m-%d")
# df_China_new_confirmed.info()
# df_China_new_confirmed.head()

In [7]:
# Reset the index as the first column
df_China_new_confirmed = df_China_new_confirmed.set_index(df_China_new_confirmed.columns[0])
# df_China_new_confirmed.head()

In [8]:
# fill N/A by 0
df_China_new_confirmed=df_China_new_confirmed.fillna(0)
# df_China_new_confirmed.head()

In [13]:
df_China_new_confirmed_final = df_China_new_confirmed.astype(int) 
# df_China_new_confirmed_final.info()
# df_China_new_confirmed_final.head()

In [10]:
# Sum the daliy confirmed patient
df_China_confirmed_final = df_China_new_confirmed_final.cumsum()
df_China_confirmed_final.tail()

,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,Henan,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-26,1423,146,68,53,128,14,110,48,63,128,...,18,47,6,21,35,5,14,11,4,0
2020-01-27,2714,188,80,66,173,23,132,72,87,168,...,33,70,8,30,46,10,19,13,6,0
2020-01-28,3554,241,91,80,296,24,147,109,121,206,...,48,99,9,37,56,13,24,15,6,0
2020-01-29,4586,311,111,101,428,27,165,162,145,278,...,65,129,14,43,63,14,26,18,6,1
2020-01-30,5806,393,132,128,537,32,206,240,178,352,...,82,168,14,59,87,17,29,20,8,1


### New Confirmed Cases of Coronavirus in Mainland China by Provincial Divisions

In [11]:
# Table of new death
df_China_new_deathes = tables[4]
df_China_new_deathes.head()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National
0,2020.01.11,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2020.01.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020.01.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020.01.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020.01.15,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [12]:
# Table of new recovered
df_China_new_recovered = tables[5]
df_China_new_recovered.head()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National
0,2020.01.11,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,2020.01.12,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,2020.01.13,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2020.01.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020.01.15,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
